In [25]:
import os
import scipy.io as sio
import numpy as np
from keras.utils.np_utils import to_categorical

import import_nbmodule
from EEGNet import EEGNet

In [26]:

def inter_pre_EEGNet(leaveonesubi):
    """
    inter_pre_EEGNet return the inter-subject prediction accuracy for leaveonesubi base on EEGNet
    
    Input:
        leaveonesubi: leave-one-subject validation in inter-subject case
        
    Output:
        

    """
    
    loadpath = os.path.join('..', '..', 'data')

    ###########################
    #
    # load train and test data
    #
    ###########################
    leaveonesubi = 1 
    x_train = np.array([])
    y_train = np.array([])
    for subi in range(1,11):  
        filename = 'alldata_sub' + str(subi) + '.mat'
        matfile = os.path.join(loadpath, filename)
        matdat = sio.loadmat(matfile, variable_names = ['eeg', 'labels_dummy', 'labels'])

        if subi == leaveonesubi:
            x_test = matdat['eeg'] # eeg: ntrials * nchns(31) * ntemporal
            y_test = matdat['labels'] # labels: ntrials * 1
        else:
            if x_train.size == 0: # x_train = [] and y_train = []
                x_train = matdat['eeg'] 
                y_train = matdat['labels']
            else:
                x_train = np.concatenate((x_train,matdat['eeg']), axis = 0)
                y_train = np.concatenate((y_train,matdat['labels']), axis = 0)
    
    
    # expand the dim of x_train, x_test
    x_train = np.expand_dims(x_train, axis = 3)  # expand x_train dim, xtrain: ntrials * nchns * ntemporal * 1
    x_test = np.expand_dims(x_test, axis = 3) # expand x_test dim, xtest: ntrials * nchns * ntemporal * 1

    # Converts a class vector (integers) to binary class matrix.
    y_train_categorical = to_categorical(y_train)
    y_test_categorical = to_categorical(y_test)

    ###########################################
    #
    # EEGNet config: F1, D, rate_dropbox, et.al
    #
    ###########################################
    F1, D = 4, 2 # EEGNet-4,2
    rate_dropout = 0.25 # paper used 0.5 for within-subject, and 0.25 for cross-subject
    fs = 256 # figured out from previous code
    input_shape = x_train[0].shape # input_shape = (nchns, ntemporal)
    n_classes = len(np.unique(y_train))

    model = EEGNet(F1 = F1, D = D, rate_dropout = rate_dropout, input_shape = input_shape, 
                   fs = fs, n_classes = n_classes)

    # training
    history = model.fit(x_train, y_train_categorical, batch_size=200, epochs=50, validation_split=0, verbose=1)
    
    # predict test
    y_pred = model.predict(x_test)

In [27]:
leaveonesubi = 1
inter_pre_EEGNet(leaveonesubi)

Epoch 1/50
15057/15057 [==============================] - 33s 2ms/step - loss: 1.1272 - acc: 0.3647
Epoch 2/50
15057/15057 [==============================] - 30s 2ms/step - loss: 1.0616 - acc: 0.4286
Epoch 3/50
15057/15057 [==============================] - 33s 2ms/step - loss: 1.0151 - acc: 0.4737
Epoch 4/50
15057/15057 [==============================] - 40s 3ms/step - loss: 0.9826 - acc: 0.4954
Epoch 5/50
15057/15057 [==============================] - 32s 2ms/step - loss: 0.9563 - acc: 0.5168
Epoch 6/50
15057/15057 [==============================] - 32s 2ms/step - loss: 0.9357 - acc: 0.5369
Epoch 7/50
15057/15057 [==============================] - 32s 2ms/step - loss: 0.9173 - acc: 0.5602
Epoch 8/50
15057/15057 [==============================] - 33s 2ms/step - loss: 0.9047 - acc: 0.5687
Epoch 9/50
15057/15057 [==============================] - 34s 2ms/step - loss: 0.8895 - acc: 0.5773
Epoch 10/50
15057/15057 [==============================] - 33s 2ms/step - loss: 0.8766 - acc: 0.5868

NameError: name 'predict' is not defined